In [1]:
import numpy as np                 # Para operaciones numéricas y manipulación de matrices
import pandas as pd                # Para análisis y manipulación de datos tabulares
import matplotlib.pyplot as plt 
   # Visualización de datos

%matplotlib inline

C:\Users\sebas\AppData\Local\Temp\ipykernel_8640\3233448815.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd                # Para análisis y manipulación de datos tabulares


In [2]:
countries = {
    "India (IN)": "IN",
    "Japón (JP)": "JP",
    "Korea del Sur (KR)": "KR",
    "Mexico (MX)": "MX",
    "Rusia (RU)": "RU"
}

data_path = "./data/" # Direccion donde se encuentran los datasets

In [3]:

df = pd.DataFrame()

dataframes = {}
for country, filename in countries.items():
    filepath = data_path + filename + "videos_cc50_202101.csv"

    cur_df = pd.read_csv(filepath)
    cur_df['Country'] = filename
    df = pd.concat([df, cur_df], ignore_index=True)

    
# Con el arreglo dataframes se puede acceder a su marcos de datos por nombre de país (por ejemplo, marcos de datos dataframes["India (IN)"]

C:\Users\sebas\AppData\Local\Temp\ipykernel_8640\570607943.py:7: DtypeWarning: Columns (4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  cur_df = pd.read_csv(filepath)


In [4]:
df.to_csv('join.csv', index=False)

In [5]:
df = pd.read_csv('join.csv')

C:\Users\sebas\AppData\Local\Temp\ipykernel_8640\3409014922.py:1: DtypeWarning: Columns (4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('join.csv')


In [6]:
df.size

3820530

In [7]:
df.isna().sum().sum() 

148612

In [8]:
print(148612 * 100 / 3820530)

3.889826804134505


In [9]:
df.drop(columns=[ 'title', 'thumbnail_link', 'description', 'tags'], inplace=True)

In [10]:
df['views'] = pd.to_numeric(df['views'], errors='coerce').astype('Int64')
df['likes'] = pd.to_numeric(df['likes'], errors='coerce').astype('Int64')
df['dislikes'] = pd.to_numeric(df['dislikes'], errors='coerce').astype('Int64')
df['comment_count'] = pd.to_numeric(df['comment_count'], errors='coerce').astype('Int64')
df['video_id'] = df['video_id'].astype('str')
df['category_id'] = pd.to_numeric(df['category_id'], errors='coerce').astype('Int64')
df['category_id'] =df['category_id'].astype('str')


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181930 entries, 0 to 181929
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   video_id                181930 non-null  object 
 1   trending_date           161997 non-null  object 
 2   channel_title           173765 non-null  object 
 3   category_id             181930 non-null  object 
 4   publish_time            173656 non-null  object 
 5   views                   173631 non-null  Int64  
 6   likes                   173631 non-null  Int64  
 7   dislikes                173631 non-null  Int64  
 8   comment_count           173631 non-null  Int64  
 9   comments_disabled       173631 non-null  object 
 10  ratings_disabled        173631 non-null  object 
 11  video_error_or_removed  173631 non-null  object 
 12  state                   181930 non-null  object 
 13  lat                     181930 non-null  float64
 14  lon                 

In [12]:
df.isna().sum()

video_id                      0
trending_date             19933
channel_title              8165
category_id                   0
publish_time               8274
views                      8299
likes                      8299
dislikes                   8299
comment_count              8299
comments_disabled          8299
ratings_disabled           8299
video_error_or_removed     8299
state                         0
lat                           0
lon                           0
geometry                      0
Country                       0
dtype: int64

In [13]:
def invalidId(row):
    if (row['video_id'].__len__() != 11 and row['video_id']!='#NAME?') or row['video_id'].count(' ') != 0 or row['video_id'].count('\\n') != 0 or row['video_id'].startswith('＠'):
        return True
    return False

In [14]:
df.loc[df.apply(invalidId, axis=1)].__len__()

8305

In [15]:
df.drop(df.loc[df.apply(invalidId, axis=1)].index, inplace=True)

In [16]:
df.duplicated().sum()

435

In [17]:
df = df.drop_duplicates()

In [18]:
df.duplicated(subset=['video_id' , 'Country', 'trending_date', 'state'], keep=False).sum()

434

In [19]:
df.drop_duplicates(subset=['video_id' , 'Country', 'trending_date', 'state'],keep='last', inplace=True)

In [20]:
df.drop(df[df['video_error_or_removed']=='VERDADERO'].index, inplace=True)

In [21]:
df.drop(columns=['video_error_or_removed'], inplace=True)

In [22]:
def convert_date(row):
    try:
        date_str = row['publish_time']
        
        if ('T' in date_str and 'Z' in date_str) or( '+' in date_str):
            utc_date = pd.to_datetime(date_str, utc=True)
        else:

            local_date = pd.to_datetime(date_str, format='%d/%m/%Y %H:%M')
            
            if row['Country'] == 'MX':
                local_date = local_date.tz_localize('America/Mexico_City')
            elif row['Country'] == 'IN':
                local_date = local_date.tz_localize('Asia/Kolkata')
            elif row['Country'] == 'JP':
                local_date = local_date.tz_localize('Asia/Tokyo')
            elif row['Country'] == 'KR':
                local_date = local_date.tz_localize('Asia/Seoul')
            elif row['Country'] == 'RU':
                local_date = local_date.tz_localize('Europe/Moscow')


            utc_date = local_date.tz_convert('UTC')
        
        return utc_date
    except Exception as e:
        print(f"Error converting date: {row['publish_time']} for country: {row['Country']}")
        print(f"Exception: {e}")
        return pd.NaT
        #print(row['publish_time'])

In [23]:
df['publish_time'] =  df.apply(convert_date, axis=1)

Error converting date: 01/04/2018 2:02 for country: MX
Exception: 2018-04-01 02:02:00
Error converting date: 01/04/2018 2:47 for country: MX
Exception: 2018-04-01 02:47:00


In [24]:
def parseToUTC(row):
    try:
        date_str = row['trending_date']
        if row['Country'] == 'JP':
            date_str = pd.to_datetime(date_str, format='%d/%m/%Y')
        else: 
            date_str = pd.to_datetime(date_str, format='%y.%d.%m')
        
        if row['Country'] == 'MX':
            local_date = date_str.tz_localize('America/Mexico_City')
        elif row['Country'] == 'IN':
            local_date = date_str.tz_localize('Asia/Kolkata')
        elif row['Country'] == 'JP':
            local_date = date_str.tz_localize('Asia/Tokyo')
        elif row['Country'] == 'KR':
            local_date = date_str.tz_localize('Asia/Seoul')
        elif row['Country'] == 'RU':
            local_date = date_str.tz_localize('Europe/Moscow')
        else:
            return pd.NaT
        
        utc_date = local_date.tz_convert('UTC')
        
        return utc_date
    except ValueError as e:
        return pd.NaT

In [25]:
df['trending_date'] = df.apply(parseToUTC, axis=1)

In [26]:
df.drop( df[df['trending_date'].isna() | df['publish_time'].isna()].index , inplace=True)

In [27]:
df[df['Country']=='JP']

,video_id,trending_date,channel_title,category_id,publish_time,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,state,lat,lon,geometry,Country
38533,5ugKfHgsmYw,2002-07-17 15:00:00+00:00,時事通信映像センター,25,2018-02-05 13:04:00+00:00,188085,591,189,0,VERDADERO,FALSO,Kyoto,35.450406,135.333331,POINT (135.3333309 35.4504059),JP
38534,ohObafdd34Y,2002-07-17 15:00:00+00:00,神谷えりな Kamiya Erina 2,1,2018-02-05 14:01:00+00:00,90929,442,88,174,FALSO,FALSO,Ishikawa,36.560002,136.640021,POINT (136.6400211 36.56000226),JP
38535,aBr2kKAHN6M,2002-07-17 15:00:00+00:00,SpaceX,28,2018-02-06 07:38:00+00:00,6408303,165892,2331,3006,FALSO,FALSO,Akita,39.709991,140.089991,POINT (140.0899914 39.70999086),JP
38536,5wNnwChvmsQ,2002-07-17 15:00:00+00:00,アシタノワダイ,25,2018-02-05 16:08:00+00:00,96255,1165,277,545,FALSO,FALSO,Kochi,33.562433,133.537523,POINT (133.5375232 33.56243329),JP
38537,B7J47qFvdsk,2002-07-17 15:00:00+00:00,シネマトゥデイ,1,2018-02-05 12:30:00+00:00,108408,1336,74,201,FALSO,FALSO,Mie,34.717066,136.516670,POINT (136.5166695 34.71706565),JP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59855,Ze2cFztLzCA,2006-12-17 15:00:00+00:00,ハイサイ探偵団,17,2018-06-10 20:00:00+00:00,72746,1069,94,158,FALSO,FALSO,Ishikawa,36.560002,136.640021,POINT (136.6400211 36.56000226),JP
59856,UFGJVHWAJaw,2006-12-17 15:00:00+00:00,ARROWS-SCREEN,24,2018-06-10 19:00:00+00:00,58863,287,325,67,FALSO,FALSO,Gifu,35.423095,136.762753,POINT (136.7627526 35.42309491),JP
59857,KWnloM9c2Hk,2006-12-17 15:00:00+00:00,釣りいろは,17,2018-06-10 20:56:00+00:00,83861,1655,48,165,FALSO,FALSO,Fukui,36.070420,136.220047,POINT (136.2200468 36.07041974),JP
59858,2SzWJ0VmsbQ,2006-12-17 15:00:00+00:00,Y's Kitchen,26,2018-06-10 21:00:00+00:00,18326,530,265,80,FALSO,FALSO,Tottori,35.500377,134.233295,POINT (134.2332946 35.50037701),JP


In [28]:
def FixDatesJapan(row):
    if row['Country'] !='JP' :
        return row['trending_date']
    date = row['trending_date']
    year= date.year
    month = date.month
    day = date.day
    hour = date.hour
    minute = date.minute
    fix_year = 2018
    fix_day = month
    fix_month = year % 100
    new_date = pd.Timestamp(year=fix_year, month=fix_month, day=fix_day, hour=hour, minute=minute, tz='UTC')
    return new_date



In [29]:
df['trending_date'] = df.apply(FixDatesJapan, axis=1)

In [30]:
import json
gender = {}

for key, value in countries.items():
    filepath = data_path + value + "_category_id.json"
    gender[value] = {}

    cur = json.load(open(filepath, 'r'))
    for i in cur['items']:
        gender[value][i['id']] = i['snippet']['title']
print(gender)



{'IN': {'1': 'Film & Animation', '2': 'Autos & Vehicles', '10': 'Music', '15': 'Pets & Animals', '17': 'Sports', '18': 'Short Movies', '19': 'Travel & Events', '20': 'Gaming', '21': 'Videoblogging', '22': 'People & Blogs', '23': 'Comedy', '24': 'Entertainment', '25': 'News & Politics', '26': 'Howto & Style', '27': 'Education', '28': 'Science & Technology', '30': 'Movies', '31': 'Anime/Animation', '32': 'Action/Adventure', '33': 'Classics', '34': 'Comedy', '35': 'Documentary', '36': 'Drama', '37': 'Family', '38': 'Foreign', '39': 'Horror', '40': 'Sci-Fi/Fantasy', '41': 'Thriller', '42': 'Shorts', '43': 'Shows', '44': 'Trailers'}, 'JP': {'1': 'Film & Animation', '2': 'Autos & Vehicles', '10': 'Music', '15': 'Pets & Animals', '17': 'Sports', '18': 'Short Movies', '19': 'Travel & Events', '20': 'Gaming', '21': 'Videoblogging', '22': 'People & Blogs', '23': 'Comedy', '24': 'Entertainment', '25': 'News & Politics', '26': 'Howto & Style', '27': 'Education', '28': 'Science & Technology', '30':

In [31]:
def setGender(row):
    try:
        return  gender[row['Country']][row['category_id']]
    except KeyError:

        return np.nan

In [32]:
df['category_name'] = df.apply(setGender, axis=1)

In [33]:
df.dropna(inplace=True)

In [34]:
df.drop(columns=['category_id', 'publish_time', 'video_id'], inplace=True)

In [35]:
def Normalize(status):
    if status == 'FALSO':
        return False
    elif status == 'VERDADERO':
        return True
    else:
        return np.nan

In [36]:
df['comments_disabled'].unique()

array(['FALSO', 'VERDADERO'], dtype=object)

In [37]:
df['comments_disabled'] = df['comments_disabled'].apply(Normalize)

In [38]:
df['ratings_disabled'] = df['ratings_disabled'].apply(Normalize)

In [39]:
df.to_csv('cleaned_data.csv', index=False)